# Descrizione notebook

In [1]:
# import delle librerie necessarie
import numpy as np
import pandas as pd
import math
import random
from itertools import combinations
from ipynb.fs.defs.creazione_CSV import normalization
from ipynb.fs.defs.UBCF import weight_factor, score_user_item, recommendation_1
from ipynb.fs.defs.IBCF import cosin_similarity, score_user_item, recommendation_2

In [2]:
# lettura della matrice con gli utenti che hanno valutato tutte le barzellette
complete_ratings = pd.read_csv('./data/complete_ratings.csv')
complete_ratings = complete_ratings.head(50)

# lettura dei rating non normalizzati dal relativo CSV (serve solo per calcolare la predizione, nella
# quale serve calcolare il rating medio dell'utente)
# non c'è bisogno di levare i 99, tanto questo df serve solo per calcolare il rating medio dell'utente target
# e si droppano le barzellette che non ha votato
ratings_df = pd.read_csv('./data/jester_jokes_ratings.csv')

# Preparazione Training set e Test set

In [3]:
test_set = []
train_set = complete_ratings.drop(['number_of_jokes_rated'], axis = 1)
# per ogni riga (<indice_riga, riga>) e per ogni barzelletta della riga si salva la
# coppia <nome_barzelletta, rating> in una lista (altrimenti si perde il riferimento al nome della barzelletta)
# poi si prende una coppia random, si salvano le informazioni nel test set, e si toglie dalla matrice completa
# quello che si è preso almeno si ottiene il training set di conseguenza
for index, row in complete_ratings.iterrows():
    lst_app = []
    for i, elem in row[2:].items():
        lst_app.append((i, elem))
    lst_pair = random.sample(lst_app, 4)
    # pair = random.choice(lst_app)
    for pair in lst_pair:
        app = {}
        app["user_id"] = int(row["user_id"])
        app[pair[0]] = pair[1]
        test_set.append(app)
        train_set.loc[train_set["user_id"] == row["user_id"], pair[0]] = np.nan
    
test_set_df = pd.DataFrame(test_set)
# test_set
# train_set

In [4]:
normalizated_train_set = normalization(train_set)

In [5]:
test_set_df

,user_id,joke_89,joke_45,joke_58,joke_48,joke_80,joke_69,joke_57,joke_9,joke_38,...,joke_12,joke_90,joke_20,joke_3,joke_44,joke_81,joke_92,joke_2,joke_64,joke_88
0,2,1.8137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,NaN,-7.0663,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,-4.8863,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,NaN,NaN,NaN,-3.6663,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,NaN,NaN,NaN,NaN,5.6277,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,194,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,195,10.8550,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Valutazione MAE

In [6]:
def mae(final, test_set): 
    t1 = 0 
    for i, j in zip(final, test_set):
        joke = ''
        for k in i.keys():
            if k.startswith('joke'):
                joke = k
        t1+=abs(i[joke] - j[joke])
    return t1 / len(final)

# Valutazione RMSE

In [7]:
def rmse(final, test_set): 
    t1 = 0 
    for i, j in zip(final, test_set):
        joke = ''
        for k in i.keys():
            if k.startswith('joke'):
                joke = k
        t1+=(i[joke] - j[joke]) * (i[joke] - j[joke])
    return np.sqrt(t1 / len(final))

# Raccomandazione UBCF

In [8]:
final = []

# per ogni utente nel training set
for n in list(train_set["user_id"]):
    lst = recommendation_1(n, normalizated_train_set, train_set)
    final = final + lst

per la barzelletta joke_45 si è predetto un punteggio di: -7.127483013780133
per la barzelletta joke_48 si è predetto un punteggio di: 3.021874404596108
per la barzelletta joke_58 si è predetto un punteggio di: -1.998186601386932
per la barzelletta joke_89 si è predetto un punteggio di: 1.820129531953189

il valore più alto predetto è 3.021874404596108 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_48
per la barzelletta joke_9 si è predetto un punteggio di: -0.5384132243321276
per la barzelletta joke_57 si è predetto un punteggio di: -3.4732472715763634
per la barzelletta joke_69 si è predetto un punteggio di: 0.3515711181425122
per la barzelletta joke_80 si è predetto un punteggio di: 0.01946105354118596

il valore più alto predetto è 0.3515711181425122 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_69
per la barzelletta joke_14 si è predetto un punteggio di: 0.8245547639145696
per la barze

per la barzelletta joke_56 si è predetto un punteggio di: 3.626545844276801
per la barzelletta joke_59 si è predetto un punteggio di: -1.8937644566645688
per la barzelletta joke_72 si è predetto un punteggio di: 3.1861006782888497

il valore più alto predetto è 3.626545844276801 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_56
per la barzelletta joke_19 si è predetto un punteggio di: -1.3836785020915738
per la barzelletta joke_36 si è predetto un punteggio di: 3.1452300021151043
per la barzelletta joke_41 si è predetto un punteggio di: -1.1593563222561711
per la barzelletta joke_87 si è predetto un punteggio di: 1.138215022082529

il valore più alto predetto è 3.1452300021151043 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_36
per la barzelletta joke_49 si è predetto un punteggio di: 2.0829895370547704
per la barzelletta joke_59 si è predetto un punteggio di: -1.6586316722367782
per la bar

per la barzelletta joke_7 si è predetto un punteggio di: -1.8450813428218678
per la barzelletta joke_33 si è predetto un punteggio di: -0.8731558184883597
per la barzelletta joke_48 si è predetto un punteggio di: 1.6226736467908625
per la barzelletta joke_49 si è predetto un punteggio di: 2.2950513913034962

il valore più alto predetto è 2.2950513913034962 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_49
per la barzelletta joke_2 si è predetto un punteggio di: 0.622330359969123
per la barzelletta joke_44 si è predetto un punteggio di: -3.613756278913332
per la barzelletta joke_64 si è predetto un punteggio di: -2.629756949912445
per la barzelletta joke_96 si è predetto un punteggio di: 0.08065358562351803

il valore più alto predetto è 0.622330359969123 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_2
per la barzelletta joke_18 si è predetto un punteggio di: -2.1634770920207265
per la barze

In [9]:
final

[{'user_id': 2, 'joke_45': -7.127483013780133},
 {'user_id': 2, 'joke_48': 3.021874404596108},
 {'user_id': 2, 'joke_58': -1.998186601386932},
 {'user_id': 2, 'joke_89': 1.820129531953189},
 {'user_id': 6, 'joke_9': -0.5384132243321276},
 {'user_id': 6, 'joke_57': -3.4732472715763634},
 {'user_id': 6, 'joke_69': 0.3515711181425122},
 {'user_id': 6, 'joke_80': 0.01946105354118596},
 {'user_id': 8, 'joke_14': 0.8245547639145696},
 {'user_id': 8, 'joke_38': -1.987583548671816},
 {'user_id': 8, 'joke_57': -4.2810337806034955},
 {'user_id': 8, 'joke_66': 0.6948095056853135},
 {'user_id': 9, 'joke_69': 3.5731700021304698},
 {'user_id': 9, 'joke_70': 0.49718517853514715},
 {'user_id': 9, 'joke_94': -0.550221659460924},
 {'user_id': 9, 'joke_95': 0.20515999884704295},
 {'user_id': 12, 'joke_26': 1.9632453280991844},
 {'user_id': 12, 'joke_52': -0.13555894684124786},
 {'user_id': 12, 'joke_56': 2.8890488485263575},
 {'user_id': 12, 'joke_79': 0.36771161300000393},
 {'user_id': 14, 'joke_5': -1.

# Raccomandazione IBCF

In [10]:
# si costruisce la matrice di similarità, ovvero una lista di coppie di item, per 
# ogni coppia si specifica la similarità (ad -1 a 1)
all_jokes = (list(train_set.columns))[1:]
jokes_pairs = list(combinations(all_jokes, 2))

similarity_matrix = []

for i_1, i_2 in jokes_pairs:
    app = normalizated_train_set[[i_1, i_2]].dropna()
    similarity_matrix.append([i_1, i_2, cosin_similarity(app[i_1], app[i_2])])
    
df = pd.DataFrame(similarity_matrix, columns = ['elem_1', 'elem_2', 'similarity'])

In [11]:
final_2 = []
for n in list(complete_ratings["user_id"]):
    lst = recommendation_2(n, normalizated_train_set, df)
    final_2 = final_2 + lst

per la barzelletta joke_45 si è predetto un punteggio di: 56.910553461721186
per la barzelletta joke_48 si è predetto un punteggio di: 193.60152442063236
per la barzelletta joke_58 si è predetto un punteggio di: -20.95478553409899
per la barzelletta joke_89 si è predetto un punteggio di: -65.37660883693431

il valore più alto predetto è 193.60152442063236 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_48
per la barzelletta joke_9 si è predetto un punteggio di: 0.998306860046478
per la barzelletta joke_57 si è predetto un punteggio di: 2.92290940278004
per la barzelletta joke_69 si è predetto un punteggio di: -5.739143761753535
per la barzelletta joke_80 si è predetto un punteggio di: 3.307926643702185

il valore più alto predetto è 3.307926643702185 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_80
per la barzelletta joke_14 si è predetto un punteggio di: -10.233031974759147
per la barzellet

per la barzelletta joke_87 si è predetto un punteggio di: -61.56658895216494

il valore più alto predetto è 251.75809840471496 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_27
per la barzelletta joke_10 si è predetto un punteggio di: 20.735585752071817
per la barzelletta joke_14 si è predetto un punteggio di: -17.154463227274437
per la barzelletta joke_50 si è predetto un punteggio di: 280.759709835825
per la barzelletta joke_51 si è predetto un punteggio di: 15.43538703563274

il valore più alto predetto è 280.759709835825 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_50
per la barzelletta joke_60 si è predetto un punteggio di: 9.082166904837603
per la barzelletta joke_70 si è predetto un punteggio di: 582.7042717405894
per la barzelletta joke_85 si è predetto un punteggio di: -0.18865264941456156
per la barzelletta joke_93 si è predetto un punteggio di: 107.41114557863516

il valore più 

per la barzelletta joke_13 si è predetto un punteggio di: 14.345682143380536
per la barzelletta joke_30 si è predetto un punteggio di: 28.715158944108502
per la barzelletta joke_80 si è predetto un punteggio di: -234.56867589731485
per la barzelletta joke_81 si è predetto un punteggio di: -16.86706700840475

il valore più alto predetto è 28.715158944108502 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_30
per la barzelletta joke_6 si è predetto un punteggio di: -83.69388830225047
per la barzelletta joke_40 si è predetto un punteggio di: 16.180846475291894
per la barzelletta joke_45 si è predetto un punteggio di: 119.21629338354967
per la barzelletta joke_76 si è predetto un punteggio di: -192.8256572036903

il valore più alto predetto è 119.21629338354967 su 10
quindi la barzelletta da raccomandare per la quale si è predetto quel punteggio è:  joke_45
per la barzelletta joke_14 si è predetto un punteggio di: -23.890829922513692
per la bar

In [12]:
len(final_2)

197

# Confronto: UBCF vs. ITCF

In [13]:
print("MAE UBCF: " + str(mae(final, test_set)))
print("RMSE UBCF: " + str(rmse(final, test_set)))

#print("MAE IBCF: " + str(mae(final_2, test_set)))
#print("RMSE IBCF: " + str(rmse(final_2, test_set)))

KeyError: 'joke_45'